<p style="color:#211f78"> 
<center><font size= 6> <b> COMPARACIÓN AUTOMATIZADA DE PRECIOS DE PRODUCTOS DE PRIMERA NECESIDAD EN LOS 3 SUPERMERCADOS MÁS POPULARES: METRO, TOTTUS y PLAZA VEA. </b> </font> </center> 
</p>

<p style="color:#63381f"><font size= 5.5> <b>Integrantes Grupo 6:</b></font> </p> 

<font size= 3.5>
    
<p style="color:#63381f">   
- Tanta Medina, Christian Rodolfo 20200338
</p> 
    
<p style="color:#63381f">
- Taquiri Pillaca, Jorge Ignacio 20200339
</p> 
    
<p style="color:#63381f">
- Tay Saccaco, Brian Eduardo 20210849
</p> 
    
</font>

## DESCRIPCIÓN DEL CASO

Se quiere automatizar la comparación de precios de productos básicos de la canasta familiar
(arroz, azúcar, aceite, leche, etc) de tres supermercados que ofrecen sus productos a través de
su propio portal web. Estos supermercados son: Metro, Plaza Vea y Tottus.
Por lo que se pide escribir un programa en Python utilizando Jupyter notebook que recopile los
precios de un producto de la canasta básica familiar de 3 Supermercados anteriormente
mencionados y reporte todos sus artículos relacionados con la palabra clave ingresada en la
búsqueda, para que posteriormente pueda comparar los precios en los distintos
Supermercados de los productos exactamente iguales. Además, que se envíe
automáticamente al correo del usuario-consultor un reporte de precio más alto, precio más
bajo, productos en común comparando los precios entre los supermercados coincidentes.

# Importar librerías


In [220]:
import pandas as pd
import math
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.microsoft import EdgeChromiumDriverManager

from tqdm import tqdm
from termcolor import colored

# Maximizar las columnas y filas
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None) 

# Ingresar producto y correo 

In [221]:
busqueda=str(input("Coloque el producto a buscar: "))  
# Ingreso del producto (por ej: Aceite Primor 1L)
# Ingreso del producto (por ej: Leche Gloria)

Coloque el producto a buscar: azucar rubia


In [173]:
para_email= input("Enviar comparación de productos al correo: ") # Email de destino

Enviar comparación de productos al correo: 20200339@lamolina.edu.pe


<p style="color:#deb03e"> <font size=6.5> <b> Programa para obtener los productos y precios de supermercados Metro (Usuario Jorge) </b> </font> </p>  

In [222]:
urls='https://www.metro.pe'          # URL de la pagina de Metro                   
options = webdriver.EdgeOptions()    # Opciones del navegador Edge
options.add_argument('--incognito')  # Opcion de abrir ventana de Edge en modo incognito
options.add_argument("--start-maximized")  # Ventana maximizada 
s = Service(EdgeChromiumDriverManager().install()) # Esta opcion nos permite usar el browser sin necesidad de colocar su ubicación                     
driver = webdriver.Edge(service=s, options = options) # Iniciar el driver con las opciones descritas anteriormente
driver.implicitly_wait(3) # Esperar 3 seg en caso de errores y si no hay saltarse ese paso y seguir con la siguiente linea     
driver.get(urls) # Abrir el navegador con la url
time.sleep(2)   # Esperar 2 segundos

In [223]:
# Para cancelar las notificaciones de Metro (ventana emergente)
WebDriverWait(driver, 25)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                      '#onesignal-slidedown-cancel-button'.replace(' ', '.'))))\
    .click()

In [224]:
# Hacer clic en "Si acepto", pagina emergente sobre "Política general de Cookies"
driver.find_element(By.XPATH, '/html/body/div[46]/div/div[3]/button[1]').click()
time.sleep(1)

In [225]:
# Hacer clic en la barra de búsqueda
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input').click()
time.sleep(1)

In [226]:
# Ingresar el producto en la barra de busqueda y realizar la busqueda (+ keys.ENTER simula el boton de ENTER)
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input').send_keys(busqueda + Keys.ENTER)
time.sleep(1)

In [227]:
# Cantidad de productos en la pagina.
page=driver.find_element(By.XPATH, '/html/body/div[27]/div/div[2]/div[1]/div[1]/span[2]').text
print(page)
#int(page[:-10])

16 productos


In [228]:
# Automatización de scrolls (se correra las veces que sea necesario) 
scrolls = 5 + 1          # Seran 5 scrolls automaticos
while True:               # Mientras esto sea verdad se realizará lo siguiente:
    scrolls -= 1          # Cuenta regresiva, se resta 1 cada ciclo
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # Se desplazará hasta la parte inferior de la página 
    time.sleep(1)         # Cada scroll automático tendra un timelap de 1 segundo
    if scrolls < 0:       # Cuando llegue a cero segundos:
        time.sleep(2)     # Le damos 2 segundos a que la pagina cargue los contenidos hasta el ultimo scroll automatico realizado
        elements = driver.find_elements(By.XPATH,'//li[@layout="19ccd66b-b568-43cb-a106-b52f9796f5cd"]') # Encontrar los productos de la pagina de acuerdo al XPATH
        break             # Se cierra el ciclo

        
print(colored('Productos extraidos:','red'))  
print(len(elements))      # Longitud de los elementos (productos extraidos) encontrados hasta coindicir con el total de productos
print('\n')
print(colored('Se llegó a extraer todos los productos?','yellow'))  
print(len(elements) == int(page[:-10])) # Comparación 

Productos extraidos:
16


Se llegó a extraer todos los productos?
True


In [229]:
# Almacenar productos y precios de Metro 
productos1 =  []   # lista vacia para productos de Metro
precios1 = []      # lista vacia para precio de Metro
supermercado1 = [] 
for element in tqdm(elements):  # tqdm:Contar iteraciones, indicar el tiempo que tarda por iteración y el tiempo total transcurrido.  
        producto = element.find_element(By.XPATH,'.//a[@class="product-item__name"]').text  # Buscar producto en base al XPATH y extraer el texto
        precio = element.find_element(By.XPATH,'.//span[@class="product-prices__value product-prices__value--best-price"]').text # Buscar precio en base al XPATH y extraer el texto
        productos1.append(producto)    # Agregar a la lista productos1
        precios1.append(precio)        # Agregar a la lista precios1
        supermercado1.append('Metro')

100%|██████████| 16/16 [00:00<00:00, 16.76it/s]


In [230]:
#Exportacion de productos y precios a un archivo excel 
dic= {'Supermercado': supermercado1 ,'Producto': productos1 , "Precio": precios1}  # Crear diccionario con los values productos y precios (listas)
df = pd.DataFrame(dic , columns= ['Supermercado', "Producto", "Precio"] ) # Convertir a dataframe con columnas de etiquetas Producto y Precio
df.to_excel('metro.xlsx', index=False,encoding='cp1252')  # Exportar a excel

# Lectura y muestra de resultados
metro = pd.read_excel('metro.xlsx')
metro

,Supermercado,Producto,Precio
0,Metro,Azúcar Rubia Paramonga 1kg,S/5.85
1,Metro,Azúcar Rubia Costeño 1 kg,S/5.90
2,Metro,Azúcar Rubia Paramonga Bolsa 5 kg,S/28.55
3,Metro,Azúcar Rubia Máxima Bolsa 1 kg,S/5.10
4,Metro,Azúcar Rubia Cuisine&Co Bolsa 1 kg,S/5.70
5,Metro,Azúcar Rubia Cuisine&Co Bolsa 5 kg,S/26.50
6,Metro,Azúcar Rubia Dulfina Bolsa 2 kg,S/10.90
7,Metro,Azúcar Rubia Máxima Bolsa 5 kg,S/23.90
8,Metro,Azúcar Rubia Metro Bolsa 5 kg,S/24.90
9,Metro,Azúcar Rubia Metro Bolsa 2 Kg,S/. 10.40


In [231]:
driver.close()  # Cerrar el driver

<p style="color:#059a38"> <font size=6.5> <b> Programa para obtener los productos y precios de supermercados Tottus (Usuario Christian) </b> </font> </p>   <font size=6.5> <b> 

In [232]:
urls='https://www.tottus.com.pe/'    # URL de la pagina Tottus
options = webdriver.EdgeOptions()    # Opciones del navegador 
options.add_argument('--incognito')  # Opcion de abrir ventana de Edge en modo incognito
options.add_argument("--start-maximized") # ventana maximizada 
s = Service(EdgeChromiumDriverManager().install()) # Esta opcion nos permite usar el driver Edge sin necesidad de colocar su ubicación                      
driver = webdriver.Edge(service=s, options = options) # Iniciar el driver con las opciones anteriores
driver.implicitly_wait(3) # Esperar 3 seg en caso de errores y si no hay saltarse ese paso
driver.get(urls) # Abrir el navegador con la url
time.sleep(2)   # Esperar 2 segundos

In [233]:
#Al iniciar y entrar a la pagina de Tottus nos salta un anuncio donde Tottus nos quiere informar sobre sus mejores ofertas,
#con este codigo automatizamos dando click en cancelar. 
WebDriverWait(driver, 25)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                      'button.align-right secondary slidedown-button'.replace(' ', '.'))))\
    .click()

In [234]:
# Cerrando ventana de mensaje de anuncios de oferta por agosto (sale de vez en cuando por ello esta comentarizada)
# driver.find_element(By.XPATH, '/html/body/div[3]/div/div/section/div[1]/div/button').click()
# time.sleep(1)

In [235]:
# Configurando click para búsqueda de productos
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/input').click()
time.sleep(1)

In [236]:
# Ingresando producto + ENTER automatico
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/input').send_keys(busqueda + Keys.ENTER)
time.sleep(1)

In [237]:
#Obtener el número de paginas
page_t = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[1]/section/div[2]/div[1]/div[2]/section/div[1]/div[1]/p').text

def page_tottus(page_t):
    if len(page_t) == 14:
        pag = int(page_t[1:2])/48  
        paginas = math.ceil(pag)  # Para paginas con menos de 10 productos
        return paginas
    elif len(page_t) == 15:
        pag = int(page_t[1:3])/48
        paginas = math.ceil(pag)  # Para paginas con menos de 100 productos
        return paginas
    elif len(page_t) == 16:
        pag = int(page_t[1:4])/48  # Para paginas con menos de 1000 productos
        paginas = math.ceil(pag)
        return paginas
    elif len(page_t) == 17:
        pag = int(page_t[1:5])/48
        paginas = math.ceil(pag)   # Para paginas con menos de 10000 productos
        return paginas
 
print(page_t)               # Total de productos
print('\n') 
print("Numero de paginas:",page_tottus(page_t))

(9 resultados)


Numero de paginas: 1


In [238]:
# Extraer todos los productos por cada pagina
productos2 = []
precios2 =[]
supermercado2 = []
n=1
while n <= paginas:
    driver.get(f'https://www.tottus.com.pe/buscar?q={busqueda}&page={n}') # Se ira iterando de acuerdo a la cantidad de paginas empezando por la pagina 1
    time.sleep(2)  # Lapso de tiempo de 2 segundos
    lista = driver.find_elements(By.XPATH,'//li[@class="product"]') # Almacenar los productos encontrados en las paginas
    for element in tqdm(lista):  
        name = element.find_element(By.XPATH,'.//h2[@class="jsx-369422875 name title mini"]').text # Extraer nombre del producto
        price = element.find_element(By.XPATH,'.//span[@class="jsx-4019956066 "]').text # Extraer el precio del producto
        productos2.append(name) 
        precios2.append(price)
        supermercado2.append('Tottus')
    n += 1


100%|██████████| 9/9 [00:00<00:00, 19.00it/s]


In [239]:
print(colored('Se llegó a extraer todos los productos?','green')) # Mensaje
print(len(productos2) >= int(paginas)) # Comparación 

Se llegó a extraer todos los productos?
True


In [240]:
# Exportación y lectura del archivo excel
dic= {'Supermercado':supermercado2 , 'Producto': productos2,"Precio": precios2 }
df = pd.DataFrame(dic , columns= ['Supermercado',"Producto","Precio"] )
df.to_excel('tottus.xlsx', index=False,encoding='cp1252')

# Lectura
tottus = pd.read_excel('tottus.xlsx')
tottus

,Supermercado,Producto,Precio
0,Tottus,AZÚCAR RUBIA DULFINA 5 KG,s/ 22.9 UN
1,Tottus,AZÚCAR RUBIA DULFINA 1 KG,s/ 5.2 UN
2,Tottus,AZÚCAR RUBIA DULFINA 2 KG,s/ 9.5 UN
3,Tottus,AZÚCAR RUBIA TOTTUS 2 KG,s/ 8.5 UN
4,Tottus,AZÚCAR RUBIA TOTTUS 1 KG,s/ 4.7 UN
5,Tottus,AZÚCAR RUBIA TOTTUS 5 KG,s/ 19.9 UN
6,Tottus,AZÚCAR RUBIA LIGHT ONZA 500 GR,s/ 9.2 UN
7,Tottus,AZÚCAR RUBIA CON STEVIA TOTTUS 500 GR,s/ 7.5 UN
8,Tottus,AZÚCAR RUBIA IN THE RAW 907 GR,s/ 23.5 UN


In [241]:
driver.close()  # Cerrar el driver

<p style="color:#fc0313"> <font size=6.5> <b> Programa para obtener los productos y precios de supermercados Plaza Vea  (Usuario Brian)  </b> </font> </p>   <font size=6.5> <b>

In [242]:
url='https://www.plazavea.com.pe'    # URL de la pagina de Plaza Vea
options = webdriver.EdgeOptions()    # opciones del navegador 
options.add_argument('--incognito')  # Opcion de abrir ventana de Edge en modo incognito
options.add_argument("--start-maximized") # ventana maximizada 
s = Service(EdgeChromiumDriverManager().install())   # Usan Edgedriver sin necesidad de colocar ubicacion                     
driver = webdriver.Edge(service=s, options = options) # Iniciar el driver con las opciones anteriores
driver.implicitly_wait(3) # Esperar 3 seg en caso de errores y si no hay saltarse ese paso
driver.get(url)
time.sleep(2)

In [243]:
#Al iniciar y entrar a la pagina salta una ventana emergente acerca de las notificaciones,
#con este codigo automatizamos dando click en cancelar. 
WebDriverWait(driver, 25)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                      '#onesignal-slidedown-cancel-button'.replace(' ', '.'))))\
    .click()

In [244]:
# Click en la barra de busqueda
driver.find_element(By.XPATH, '/html/body/div[3]/header/div[2]/div/div[4]/section/div[2]/div/div/input').click()

In [245]:
# Realizar busqueda y ENTER 
driver.find_element(By.XPATH, '/html/body/div[3]/header/div[2]/div/div[4]/section/div[2]/div[2]/div[1]/input').send_keys(busqueda + Keys.ENTER)

In [248]:
# Encontrando el total de productos y el numero de paginas
page_pv=driver.find_element(By.XPATH, '/html/body/div[4]/main/div/div[1]/div[2]/div[3]/div/div[1]/div/h2/span[2]').text
def page_plazavea(page_pv):
    if len(page_pv) == 13:
        pag = int(page_pv[1:2])/49.5 # para paginas con productos menores a 10
        paginas = math.ceil(pag)
        return paginas
    elif len(page_pv) == 14:
        pag = int(page_pv[1:3])/49.5  # para paginas con productos menores a 100
        paginas = math.ceil(pag)
        return paginas
    elif len(page_pv) == 15:
        pag = int(page_pv[1:4])/49.5 # para paginas con productos menores a 1000
        paginas = math.ceil(pag)
        return paginas
    elif len(page_pv) == 16:
        pag = int(page_pv[1:5])/49.5 # para paginas con productos menores a 10000
        paginas = math.ceil(pag)
        return paginas
        
print(page_pv)               # Total de productos
print('\n') 
print("Numero de paginas:",page_plazavea(page_pv))

(17 productos)


Numero de paginas: 1


In [249]:
# Extrayendo los productos y sus precios
productos3 = []
precios3 =[]
supermercado3 = []
n=1

while n <= paginas:
    driver.find_element(By.CSS_SELECTOR, f'#category-page > div.vitrine.resultItemsWrapper > div > div.paginator-container > div > div > span:nth-child({n})').click()
    time.sleep(3)
    lista = driver.find_elements(By.XPATH,'//div[@data-stock="true"]')
    for element in tqdm(lista):
        name = element.find_element(By.XPATH,'.//a[@class="Showcase__name"]').text
        price = element.find_element(By.XPATH,'.//span[@class="price"]').text
        productos3.append(name)
        precios3.append(price)
        supermercado3.append('Plaza Vea')
    n += 1   

100%|██████████| 17/17 [00:02<00:00,  5.68it/s]


In [250]:
print(colored('Se llegó a extraer todos los productos?','red'))  
print(len(productos3) >= int(paginas)) # Comparación 

Se llegó a extraer todos los productos?
True


In [251]:
# Exportar a excel
dic= {'Supermercado':supermercado3 , 'Producto': productos3,"Precio": precios3 }
df = pd.DataFrame(dic , columns= ['Supermercado',"Producto","Precio"] )
df.to_excel('plazavea.xlsx', index=False,encoding='cp1252')
# Lectura
plazavea=pd.read_excel('plazavea.xlsx')
plazavea

,Supermercado,Producto,Precio
0,Plaza Vea,Azúcar Rubia BELL'S Bolsa 4Kg,S/ 16.70 un
1,Plaza Vea,Azúcar Rubia BELL'S Selección Bolsa 5Kg,S/ 23 un
2,Plaza Vea,Azúcar Rubia BELL'S Bolsa 1.3Kg,S/ 5.70 un
3,Plaza Vea,Azúcar Rubia DULFINA Bolsa 5Kg,S/ 21.90 un
4,Plaza Vea,Azúcar Rubia BELL'S Bolsa 5Kg,S/ 19.90 un
5,Plaza Vea,Azúcar Rubia BELL'S Bolsa 650g,S/ 3.10 un
6,Plaza Vea,Azúcar Rubia BELL'S Selección Bolsa 1K,S/ 5.30 un
7,Plaza Vea,Azúcar Rubia COSTEÑO Bolsa 1Kg,S/ 5.90 un
8,Plaza Vea,Azúcar Rubia DULFINA Bolsa 1Kg,S/ 4.89 un
9,Plaza Vea,Azúcar Rubia CARTAVIO Bolsa 1Kg,S/ 4.80 un


In [203]:
driver.close() # Cerrar el driver

# COMPARACIÓN

Filtrar por la marca y cantidad del producto

In [253]:
marca = input()  # Colocar la marca, (por ejemplo Gloria, Primor)

DULFINA


In [254]:
cantidad = input() # Colocar la cantidad (sin unidades)

5


# En Metro

In [255]:
def convert1(marca):
    para_metro = marca.capitalize()  # Convertir la primera palabra en mayuscula
    return para_metro

df_metro = metro[metro['Producto'].str.count(f'(.*{convert1(marca)}.*{cantidad}.*)')==1].sort_values(by='Precio', ascending = True) # Ordenamos los resultados de menor a mayor
df_metro=df_metro.reset_index(drop=True) # Resetear los indices (index)
df_metro # Visualizar dataframe df_metro

,Supermercado,Producto,Precio
0,Metro,Azúcar Rubia Dulfina Bolsa 5 kg,S/26.45


# En Tottus

In [256]:
def convert2(marca):
    para_tottus_pv = marca.upper()  # Convertir toda la palabra en mayuscula
    return para_tottus_pv

df_tottus = tottus[tottus['Producto'].str.count(f'(.*{convert2(marca)}.*{cantidad}.*)')==1].sort_values(by='Precio', ascending = True)# Ordenamos los resultados de menor a mayor
df_tottus=df_tottus.reset_index(drop=True) # Resetear los indices (index)
df_tottus # Visualizar dataframe df_tottus

,Supermercado,Producto,Precio
0,Tottus,AZÚCAR RUBIA DULFINA 5 KG,s/ 22.9 UN


# En Plaza Vea

In [257]:
def convert3(marca):
    para_tottus_pv = marca.upper()  # Convertir toda la palabra en mayuscula
    return para_tottus_pv

df_plazavea= plazavea[plazavea['Producto'].str.count(f'(.*{convert3(marca)}.*{cantidad}.*)')==1].sort_values(by='Precio', ascending = True) # Ordenamos los resultados de menor a mayor
df_plazavea = df_plazavea.reset_index(drop=True)  # Resetear los indices (index)
df_plazavea # Visualizar dataframe df_plazavea

,Supermercado,Producto,Precio
0,Plaza Vea,Azúcar Rubia DULFINA Bolsa 5Kg,S/ 21.90 un


# Finalmente

In [258]:
comparacion = pd.concat([df_metro,df_tottus,df_plazavea]) # Concatenamos los 3 dataframes (df_metro,df_tottus,df_plazavea)
# Exportando a csv 
comparacion.to_csv('Reporte.csv', index=False,encoding='cp1252') 
# Visualizar comparación
comparacion

,Supermercado,Producto,Precio
0,Metro,Azúcar Rubia Dulfina Bolsa 5 kg,S/26.45
0,Tottus,AZÚCAR RUBIA DULFINA 5 KG,s/ 22.9 UN
0,Plaza Vea,Azúcar Rubia DULFINA Bolsa 5Kg,S/ 21.90 un


In [259]:
# Exportando a excel las comparaciones 3 hojas distintas para enviar al correo
writer = pd.ExcelWriter('Reporte.xlsx') # Reporte.xlsx con opcion a escritura

df_metro.to_excel(writer, 'Metro', index=False) # Guardamos el dataframe df_metro en una hoja llamada Metro
df_tottus.to_excel(writer, 'Tottus', index=False) # Guardamos el dataframe df_tottus en una hoja llamada Tottus
df_plazavea.to_excel(writer, 'Plaza Vea', index=False) # Guardamos el dataframe df_plazavea en una hoja llamada Plaza Vea

writer.save() # Guardar los cambios
writer.close() # Cerrar 

# Envío de correo automático

In [260]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
 
remitente = 'jorgeigna.2911@gmail.com' 
destinatarios = para_email                  # Destinatario (se colocó en un principio)
asunto = '[RESULTADOS] TRABAJO FINAL LP2 GRUPO 6'
cuerpo = '''
Hola estimado usuario, le enviamos las comparaciones de los productos.
Integrantes:
- Tanta Medina, Christian Rodolfo 20200338
- Taquiri Pillaca, Jorge Ignacio 20200339
- Tay Saccaco, Brian Eduardo 20210849
'''
ruta_adjunto = 'Reporte.xlsx'
nombre_adjunto = 'Comparacion de productos.xlsx'

mensaje = MIMEMultipart() # Creamos el objeto mensaje
# Establecemos los atributos del mensaje
mensaje['From'] = remitente
mensaje['To'] = ", ".join(destinatarios)  # . join en caso haya mas de 1 destinatario
mensaje['Subject'] = asunto

# Agregamos el cuerpo del mensaje como objeto MIME de tipo texto
mensaje.attach(MIMEText(cuerpo, 'plain')) 
# Abrimos el archivo que vamos a adjuntar
archivo_adjunto = open(ruta_adjunto, 'rb') # "rb" abre el archivo en formato binario para lectura
adjunto_MIME = MIMEBase('application', 'octet-stream') # Creamos un objeto MIME base 
adjunto_MIME.set_payload((archivo_adjunto).read()) # Cargamos el archivo adjunto
encoders.encode_base64(adjunto_MIME) # Codificamos el objeto en BASE64
adjunto_MIME.add_header('Content-Disposition', "attachment; filename= %s" % nombre_adjunto) # Agregamos una cabecera al objeto
mensaje.attach(adjunto_MIME) # Finalmente lo agregamos al mensaje

# Creamos la conexión con el servidor
sesion_smtp = smtplib.SMTP('smtp.gmail.com', 587) 
sesion_smtp.starttls() # Ciframos la conexión
sesion_smtp.login('jorgeigna.2911@gmail.com','bnriqogwriiscvoi') # Iniciamos sesión en el servidor
texto = mensaje.as_string() # Convertimos el objeto mensaje a texto
# Enviamos el mensaje
sesion_smtp.sendmail(remitente, destinatarios, texto)
# Cerramos la conexión
sesion_smtp.quit()  

(221,
 b'2.0.0 closing connection bj18-20020a0561220e5200b00378d82b16f3sm6778582vkb.54 - gsmtp')

# Envio

![foto](captura1.png "")

[](usuario.png)